In [ ]:
#!pip install q keras==2.2.4
#!pip install tensorflow==1.14.0
!pip install q keras==2.13.1
!pip install tensorflow==2.13.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [ ]:
!nvidia-smi

In [ ]:
##    Obtenemos los .npy de google drive.

!gdown 1pbyR5AMrnoJMNKLFpB95_6DlwyU3URx0
!gdown 1JgZTk7PzjeA-MZ6Ow8JGrhMU9YByfmvo

#https://drive.google.com/file/d/1pbyR5AMrnoJMNKLFpB95_6DlwyU3URx0/view?usp=sharing
#https://drive.google.com/file/d/1JgZTk7PzjeA-MZ6Ow8JGrhMU9YByfmvo/view?usp=sharing


In [ ]:
X_raw = np.load('X.npy')
#X_raw = X_raw/255
print(len(X_raw))
print(len(X_raw[1]))
print(len(X_raw[1][1]))
print(X_raw[1][1][1])

### Preparar datos

In [ ]:
X_raw = np.load('X.npy')
X_raw = X_raw/255.
y = np.load('y.npy')
X = []
size = (28,28)
for x in X_raw:
    X.append(resize(x, size))
X = np.array(X)
#X[14]

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito")
plt.ylabel("Cantidad")
plt.xlabel("Día de la semana")
plt.hist(y, bins=10)
plt.plot()
y

### Visualizando el `dataset`

In [ ]:
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    idx = np.random.choice(X.shape[0], 1)[0]
    plt.title('id:{} val:{}'.format(idx, y[idx] ))
    plt.imshow(-X[idx], cmap='gray')
    plt.axis('off')
plt.show()

### Split de train-test

In [ ]:
X.shape

In [ ]:
#Convertimos y a un array de números
print(y.shape[0])
print(y[2])
y_n = np.empty((y.shape[0]))

for i in range(y.shape[0]):
    if y[i] == 'Lunes': y_n[i] = 0;
    if y[i] == 'Martes': y_n[i] = 1;
    if y[i] == 'Miercoles': y_n[i] = 2;
    if y[i] == 'Jueves': y_n[i] = 3;
    if y[i] == 'Viernes': y_n[i] = 4;
    if y[i] == 'Sabado': y_n[i] = 5;
    if y[i] == 'Domingo': y_n[i] = 6;

y_n


In [ ]:
#Separamos el dataset en un grupo de entrenamiento y uno de testeo.
X_train, X_test, y_train, y_test = train_test_split(X, y_n, test_size=0.20, random_state=42, stratify=y)

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito (train)")
plt.ylabel("Cantidad")
plt.xlabel("Día de la semana")
plt.hist(y_train, bins=10)
plt.plot()

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito (test)")
plt.ylabel("Cantidad")
plt.xlabel("Día de la semana")
plt.hist(y_test, bins=10)
plt.plot()

In [ ]:
# Ajustamos las dimensiones de las imágenes
print(X_train.shape, X_test.shape)
if X_train.ndim == 3:
    X_train = X_train[...,None]
    X_test = X_test[..., None]
    print(X_train.shape, X_test.shape)

## Definimos un modelo

In [ ]:
#from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D, Flattenf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD

bs = 16
lr = 0.0005

#Espacio de busqueda
#Cantidad Filtros: 32, 64, 128, 256
#Tamaño del Filtro: 3, 5, 7
#Activaciones: 'relu', 'sigmoid', 'tanh'
#Unidades Dense: 128, 256, 100..
#lr: 0.001, 0.0001, 0.00001, etc
#bs: 8, 16, 32...

model = Sequential([Conv2D(32, 3, activation='relu', input_shape=(*size,1)),
                    MaxPool2D(),
                    Conv2D(64, 3, activation='relu', padding ='same'),
                    MaxPool2D(),
                    Conv2D(128, 3, activation='relu', padding ='same'),
                    MaxPool2D(),
                    Flatten(),
                    Dense(128, activation='relu'), #modificar!
                    Dense(7, activation='softmax')]) #no modificar

optimizer1=SGD(learning_rate=lr)
model.compile(optimizer= optimizer1, loss='sparse_categorical_crossentropy' , metrics=['accuracy'])
# La pérdida 'sparse_categorical_crossentropy' realiza automaticamente la conversión
# del target a one-hot encoding.

model.summary()

## Entrenamiento

In [ ]:
log = model.fit(X_train, y_train, batch_size=bs, epochs=400, validation_data=(X_test, y_test))

In [ ]:
def show_results(model, log):
    loss, acc = model.evaluate(X_test, y_test, batch_size=512, verbose=False)
    print(f'Loss     = {loss:.4f}')
    print(f'Accuracy = {acc:.4f}')
    val_loss = log.history['val_loss']
    val_acc = log.history['val_accuracy']
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(val_loss, label='test')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['accuracy'], label='train')
    ax2.plot(val_acc, label='test')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('acc')
    for ax in axes: ax.legend()

In [ ]:
show_results(model, log)

## Probando el modelo

In [ ]:
idx = np.random.choice(X_test.shape[0], 1)[0]
im = X_test[idx]
label = y_test[idx]

days = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

label_n = days[round(label)]

plt.title('(test) id:{} val:{}'.format(idx, label_n ))
plt.axis('off')
plt.imshow(-im[:,:,0], cmap='gray')
plt.show()

## Usando el modelo entrenado

In [ ]:
salida = model.predict(im[None,:,:,:])[0]
salida
#im[None,:,:,:]

In [ ]:
plt.figure()
plt.title("Predicción: {} Label: {}".format(salida.argmax(), days[round(label)] ))
plt.ylabel("Probabilidad")
plt.xlabel("Día")
plt.ylim([0,1])
#plt.bar(np.arange(7), salida)
plt.bar(days, salida)
plt.plot()